# This notebook will run through a list of samples and use single instrument settings to get either annular 1D curve or I vs q curve.

## First Method: a JSON file is imported

- Reduction parameters common to all samples are defined in a JSON file.
- Reduction parameters pertaining to the event runs are updated with a python dictionary.

In [ ]:
#USER Input here with scan numbers etc.
samples = ['9166', '9167', '9176']
samples_trans = ['9178', '9179', '9188']
sample_thick = ['0.1'] * 3
bkgd = ['9165', '9165', '9165']
bkgd_trans = ['9177'] * 3

sample_names=["Al4", "PorasilC3", "PTMA-15"]

### From this point on, user should not make any changes

In [ ]:
%matplotlib inline
import json
import os
from pprint import pprint as pretty_print
import time
from drtsans.mono.gpsans import (load_all_files, reduce_single_configuration, plot_reduction_output,
                                 reduction_parameters, update_reduction_parameters, validate_reduction_parameters)

In [ ]:
# reduction parameters common to all the runs for this notebook
json_file = '/HFIR/CG2/shared/sans-backend/notebooks/gpsans/gpsans_reduction_1config.json'
with open(json_file, 'r') as file_handle:
    common_configuration_full = json.load(file_handle)

In [ ]:
# I'm working of a better printout of the reduction parameters. For now you can use pretty_print
pretty_print(common_configuration_full)

In [ ]:
# Create output directory

output_dir = common_configuration_full['configuration']['outputDir']
for subfolder in ['1D','2D']:
    output_folder=os.path.join(output_dir,subfolder)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

print('Output directory is "', output_dir, '"', sep='')

In [ ]:
# reduce each run

start_time = time.time()
for i in range(len(samples)):
    # Settings particular to each reduction session
    run_data = {
        'sample': {
            'runNumber': samples[i],
            'thickness': sample_thick[i],
            'transmission': {'runNumber': samples_trans[i]}
        },
        'background': {
            'runNumber': bkgd[i],
            'transmission': {'runNumber': bkgd_trans[i]}
        },
        'outputFileName': sample_names[i],
    }

    # Update our common settings with the particulars of the current reduction
    reduction_input = update_reduction_parameters(common_configuration_full, run_data, validate=True)

    # Begin redution. Be sure to validate the parameters before.
    # We validated the parameters when we updated them. Otherwise you can invoke the validation like this:
    # reduction_input = validate_reduction_parameters(reduction_input)

    loaded=load_all_files(reduction_input)
    out=reduce_single_configuration(loaded, reduction_input)
    plot_reduction_output(out, reduction_input, loglog=False)

    # Save the reduction parameters of each reduction session to a JSON file
    output_dir = reduction_input['configuration']['outputDir']
    output_json_file = os.path.join(output_dir, f'{sample_names[i]}.json')  # full path to the JSON file
    with open(output_json_file, 'w') as file_handle:
        json.dump(reduction_input, file_handle, indent=2)

end_time = time.time()
print(end_time - start_time)     


In [ ]:
%%bash
head  /tmp/gpsans_reduction_1config/Al4.json

## Second Method: no input JSON file is used

- Reduction parameters common to all samples are defined in a python dictionary.
- Reduction parameters pertaining to the event runs are updated with a python dictionary.

In [ ]:
#USER Input here with scan numbers etc.
samples = ['9166', '9167', '9176']
samples_trans = ['9178', '9179', '9188']
sample_thick = ['0.1'] * 3
bkgd = ['9165', '9165', '9165']
bkgd_trans = ['9177'] * 3

sample_names=["Al4", "PorasilC3", "PTMA-15"]

### From this point on, user should not make any changes

In [ ]:
%matplotlib inline
import json
import os
from pprint import pprint as pretty_print
import time
from drtsans.mono.gpsans import (load_all_files, reduce_single_configuration, plot_reduction_output,
                                 reduction_parameters, update_reduction_parameters, validate_reduction_parameters)

In [ ]:
# reduction parameters common to all the runs for this notebook
common_configuration = {
    "iptsNumber": 21981,
    "emptyTransmission": {"runNumber": 9177},
    "beamCenter": {"runNumber": 9177},
    "configuration": {
        "outputDir": "/tmp/gpsans_reduction_1config",
        "darkFileName": "/HFIR/CG2/shared/Lilin/DC/CG2_8844.nxs.h5",
        "sensitivityFileName": "/HFIR/CG2/shared/drt_sensitivity/sens_c486_LDS.nxs",
        "DBScalingBeamRadius": 25,
        "mmRadiusForTransmission": 40,
        "absoluteScaleMethod": "direct_beam",
        "numQxQyBins": 192,
        "1DQbinType": "scalar",
        "QbinType": "log",
        "LogQBinsPerDecade": 33,
        "useLogQBinsDecadeCenter": True,
    }
}

# Add on the other reduction parameters with their default values (most will be empty)
common_configuration_full = reduction_parameters(common_configuration, 'GPSANS', validate=False)

In [ ]:
with open('/tmp/gpsans_reduction_1config.json', 'w') as file_handle:
    json.dump(common_configuration_full, file_handle, indent=2)

(the rest is the same as in the first method)

In [ ]:
# I'm working of a better printout of the reduction parameters. For now you can use pretty_print
pretty_print(common_configuration_full)

In [ ]:
# Create output directory

output_dir = common_configuration_full['configuration']['outputDir']
for subfolder in ['1D','2D']:
    output_folder=os.path.join(output_dir,subfolder)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

print('Output directory is "', output_dir, '"', sep='')

In [ ]:
# reduce each run

start_time = time.time()
for i in range(len(samples)):
    # Settings particular to each reduction session
    run_data = {
        'sample': {
            'runNumber': samples[i],
            'thickness': sample_thick[i],
            'transmission': {'runNumber': samples_trans[i]}
        },
        'background': {
            'runNumber': bkgd[i],
            'transmission': {'runNumber': bkgd_trans[i]}
        },
        'outputFileName': sample_names[i],
    }

    # Update our common settings with the particulars of the current reduction
    reduction_input = update_reduction_parameters(common_configuration_full, run_data, validate=True)

    # Begin redution. Be sure to validate the parameters before.
    # We validated the parameters when we updated them. Otherwise you can invoke the validation like this:
    # reduction_input = validate_reduction_parameters(reduction_input)

    loaded=load_all_files(reduction_input)
    out=reduce_single_configuration(loaded, reduction_input)
    plot_reduction_output(out, reduction_input, loglog=False)

    # Save the reduction parameters of each reduction session to a JSON file
    output_dir = reduction_input['configuration']['outputDir']
    output_json_file = os.path.join(output_dir, f'{sample_names[i]}.json')  # full path to the JSON file
    with open(output_json_file, 'w') as file_handle:
        json.dump(reduction_input, file_handle, indent=2)

end_time = time.time()
print(end_time - start_time)     


In [ ]:
%%bash
head  /tmp/gpsans_reduction_1config/Al4.json